# NLP

In [1]:
# Initial imports
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from pathlib import Path
import re
nltk.download('vader_lexicon')
nltk.download('stopwords')
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/tamobee/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tamobee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Display all the text in columns
pd.set_option('display.max_colwidth',None)

In [3]:
# Read in the tweets csv as a pandas DataFrame
file_name='clean_elon_tweets_vF.csv'
file_path = Path(f"../Resources/{file_name}")
tweets_data = pd.read_csv(file_path,parse_dates=True, infer_datetime_format=True)

# convert NaNs into blanks 
tweets_data = tweets_data.replace(np.nan, '', regex=True)

tweets_data.tail(5)

,date,tweet
2654,2021-01-13,"@NASASpaceflight Sea shanty tiktok takes it to a new level with actual pirate ships @ConnectDotsToo Sure Today at SpaceX is about practicing Starship engine starts. Ship is held down by massive pins while engines are fired. Two starts completed, about to try a third. @Tesmanian_com As promised Legalize comedy @lvladimirovBG You can steal our name/logos &amp; we probably won’t sue you @arstechnica @SciGuySpace We’re just trying to get people to Mars. Help would be appreciated. @lexfridman Suicide is more than double homicide https://t.co/MvWGPs9uQE @lexfridman Absolutely Hey you … Yeah you Queen … You’re gonna make it! 💕💕 https://t.co/LrqdIrbIyd @TheOldManPar @DJSnM @MachinePix Hate to say it, but might be true"
2655,2021-01-14,"@skorusARK Prototypes are easy, volume production is hard, positive cash flow is excruciating @Tesla Physics @Erdayastronaut Detanking &amp; inspections now. Good progress towards our “Hop in &amp; go to Mars!” goal. All three static fires completed &amp; no RUDs! @justpaulinelol @Erdayastronaut @SpaceX Wow, a lot has happened in 10 years! @OfficialJlipper Fair enough haha https://t.co/ho7yGXAS3a"
2656,2021-01-15,"@SuperclusterHQ @w00ki33 Fallout New Texas @Breedlove22 @benmezrich Only Chuck Norris can divide by zero @Cerberu21014829 @Breedlove22 @benmezrich Good point @Breedlove22 @benmezrich The thing we call money is just an information system for labor allocation. What actually matters is making goods &amp; providing services. We should look at currencies from an information theory standpoint. Whichever has least error &amp; latency will win. Monty Python is amazing https://t.co/UJq94IWT88 @RationalEtienne @tobyliiiiiiiiii @Erdayastronaut They sure can twist the knife in that show! @tobyliiiiiiiiii @Erdayastronaut Probably wise @Erdayastronaut We’re making major improvements to ease of engine swap. Needs to be a few hours at most. @PPathole @johnkrausphotos @SpaceX Two of the engines need slight repairs, so will be switched out @johnkrausphotos @SpaceX Nice shot @realOmarAbdalah We don’t have high school internships, but please apply when you’re in college!"
2657,2021-01-19,"@kellyreid The rate-limiting part or process in cell production is constantly changing @FrancisSuarez @CityofMiami Cars &amp; trucks stuck in traffic generate megatons of toxic gases &amp; particulate, but @boringcompany road tunnels under Miami would solve traffic &amp; be an example to the world. Spoke with @RonDeSantisFL about tunnels last week. If Governor &amp; Mayor want this done, we will do it. @RationalEtienne @OwenSparks_ @Tesla With our giant casting machines, we are literally trying to make full-size cars in the same way that toy cars are made @lexfridman @tegmark Tegmark is an exceptionally smart &amp; good human @OwenSparks_ The best manufacturing technology is in ultra high volume industries, like food &amp; beverage, some medical (eg syringes) &amp; toys Battery cell production is the fundamental rate-limiter slowing down a sustainable energy future. Very important problem. @p_ferragu Looking into this. No question that FSD should be viewed as reasonably valuable when doing a trade-in. @tobyliiiiiiiiii @Erdayastronaut For sure @Erdayastronaut Glad you’re ok @Virgin_Orbit Congratulations! @teslaownersSV This is a good one @MrBeastYT I whistle @DMC_Ryan @C_R_H_M @Tesla Cybertruck doesn’t need a garage @DMC_Ryan @Tesla It will be awesome @owenshift Good point @signalapp Your server-side code is doing too much @Wikipedia Happy birthday Wikipedia! So glad you exist. @TheOnion Guess you been watching Cobra Kai @DMC_Ryan @C_R_H_M @Tesla Cybertruck doesn’t need a garage @DMC_Ryan @Tesla It will be awesome @owenshift Good point @signalapp Your server-side code is doing too much @Wikipedia Happy birthday Wikipedia! So glad you exist. @TheOnion Guess you been watching Cobra Kai"
2658,2021-01-20,


In [4]:
# Functions for cleaning the raw tweet data
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt

def clean_tweets(tweets):
    #remove twitter Return handles (RT @xxx:)
    tweets = np.vectorize(remove_pattern)(tweets, "RT @[\w]*:")
    
    #remove twitter handles (@xxx)
    tweets = np.vectorize(remove_pattern)(tweets, "@[\w]*")
    
    #remove URL links (httpxxx)
    tweets = np.vectorize(remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
    
    #remove special characters, numbers, punctuations (except for #)
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
    
    return tweets

In [5]:
# Create a column for cleaned tweet to original DataFrame
tweets_data['cleaned_tweet'] = clean_tweets(tweets_data['tweet'])
tweets_data.tail(5)

,date,tweet,cleaned_tweet
2654,2021-01-13,"@NASASpaceflight Sea shanty tiktok takes it to a new level with actual pirate ships @ConnectDotsToo Sure Today at SpaceX is about practicing Starship engine starts. Ship is held down by massive pins while engines are fired. Two starts completed, about to try a third. @Tesmanian_com As promised Legalize comedy @lvladimirovBG You can steal our name/logos &amp; we probably won’t sue you @arstechnica @SciGuySpace We’re just trying to get people to Mars. Help would be appreciated. @lexfridman Suicide is more than double homicide https://t.co/MvWGPs9uQE @lexfridman Absolutely Hey you … Yeah you Queen … You’re gonna make it! 💕💕 https://t.co/LrqdIrbIyd @TheOldManPar @DJSnM @MachinePix Hate to say it, but might be true","Sea shanty tiktok takes it to a new level with actual pirate ships Sure Today at SpaceX is about practicing Starship engine starts. Ship is held down by massive pins while engines are fired. Two starts completed, about to try a third. As promised Legalize comedy You can steal our name/logos &amp; we probably won’t sue you We’re just trying to get people to Mars. Help would be appreciated. Suicide is more than double homicide Absolutely Hey you … Yeah you Queen … You’re gonna make it! 💕💕 Hate to say it, but might be true"
2655,2021-01-14,"@skorusARK Prototypes are easy, volume production is hard, positive cash flow is excruciating @Tesla Physics @Erdayastronaut Detanking &amp; inspections now. Good progress towards our “Hop in &amp; go to Mars!” goal. All three static fires completed &amp; no RUDs! @justpaulinelol @Erdayastronaut @SpaceX Wow, a lot has happened in 10 years! @OfficialJlipper Fair enough haha https://t.co/ho7yGXAS3a","Prototypes are easy, volume production is hard, positive cash flow is excruciating Physics Detanking &amp; inspections now. Good progress towards our “Hop in &amp; go to Mars!” goal. All three static fires completed &amp; no RUDs! Wow, a lot has happened in 10 years! Fair enough haha"
2656,2021-01-15,"@SuperclusterHQ @w00ki33 Fallout New Texas @Breedlove22 @benmezrich Only Chuck Norris can divide by zero @Cerberu21014829 @Breedlove22 @benmezrich Good point @Breedlove22 @benmezrich The thing we call money is just an information system for labor allocation. What actually matters is making goods &amp; providing services. We should look at currencies from an information theory standpoint. Whichever has least error &amp; latency will win. Monty Python is amazing https://t.co/UJq94IWT88 @RationalEtienne @tobyliiiiiiiiii @Erdayastronaut They sure can twist the knife in that show! @tobyliiiiiiiiii @Erdayastronaut Probably wise @Erdayastronaut We’re making major improvements to ease of engine swap. Needs to be a few hours at most. @PPathole @johnkrausphotos @SpaceX Two of the engines need slight repairs, so will be switched out @johnkrausphotos @SpaceX Nice shot @realOmarAbdalah We don’t have high school internships, but please apply when you’re in college!","Fallout New Texas Only Chuck Norris can divide by zero Good point The thing we call money is just an information system for labor allocation. What actually matters is making goods &amp; providing services. We should look at currencies from an information theory standpoint. Whichever has least error &amp; latency will win. Monty Python is amazing They sure can twist the knife in that show! Probably wise We’re making major improvements to ease of engine swap. Needs to be a few hours at most. Two of the engines need slight repairs, so will be switched out Nice shot We don’t have high school internships, but please apply when you’re in college!"
2657,2021-01-19,"@kellyreid The rate-limiting part or process in cell production is constantly changing @FrancisSuarez @CityofMiami Cars &amp; trucks stuck in traffic generate megatons of toxic gases &amp; particulate, but @boringcompany road tunnels under Miami would solve traffic &amp; be an example to the world. Spoke with @RonDeSantisFL about tunnels last week. 

In [6]:
# Declare variables for scores
scores = []
compound_list = []
positive_list = []
negative_list = []
neutral_list = []

# Create the sentiment scores DataFrame for Cleaned Tweets
for i in range(tweets_data['cleaned_tweet'].shape[0]):
    try:
        compound = analyzer.polarity_scores(tweets_data['cleaned_tweet'][i])["compound"]
        pos = analyzer.polarity_scores(tweets_data['cleaned_tweet'][i])["pos"]
        neu = analyzer.polarity_scores(tweets_data['cleaned_tweet'][i])["neu"]
        neg = analyzer.polarity_scores(tweets_data['cleaned_tweet'][i])["neg"]
    
        scores.append({
            "Compound": compound, 
            "Positive": pos, 
            "Negative": neg, 
            "Neutral": neu
        })
    except AttributeError:
        pass

sentiment_scores = pd.DataFrame(scores)
sentiment_scores.head(5)

,Compound,Positive,Negative,Neutral
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [7]:
# Describe the Tweet sentiment
sentiment_scores.describe()

,Compound,Positive,Negative,Neutral
count,2659.000000,2659.000000,2659.000000,2659.000000
mean,0.298149,0.105044,0.029572,0.431761
std,0.448107,0.148586,0.056029,0.395964
min,-0.959400,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.617000
75%,0.784500,0.182000,0.046500,0.789000
max,0.999700,1.000000,0.756000,1.000000


In [8]:
# Join Tweets DataFrame and sentiment scores DataFrame
#scores_df = pd.DataFrame.from_dict(scores)
tweets_data = tweets_data.join(sentiment_scores)
tweets_data.set_index('date',inplace=True)
tweets_data.head(5)

,tweet,cleaned_tweet,Compound,Positive,Negative,Neutral
date,,,,,,
2010-06-29,,,0.0,0.0,0.0,0.0
2010-06-30,,,0.0,0.0,0.0,0.0
2010-07-01,,,0.0,0.0,0.0,0.0
2010-07-02,,,0.0,0.0,0.0,0.0
2010-07-06,,,0.0,0.0,0.0,0.0


In [9]:
# Add the Sentiment column to the Tweets DataFrame

i = 0

Sentiment = []

while(i<len(tweets_data)):
    if ((tweets_data.iloc[i]['Compound'] > 0.5)):
        Sentiment.append('positive')
        i = i+1
    elif ((tweets_data.iloc[i]['Compound'] > -0.5) & (tweets_data.iloc[i]['Compound'] <= 0.5)):
        Sentiment.append('neutral')
        i = i+1
    elif ((tweets_data.iloc[i]['Compound'] <= -0.5)):
        Sentiment.append('negative')
        i = i+1

tweets_data['Sentiment'] = Sentiment
tweets_data

,tweet,cleaned_tweet,Compound,Positive,Negative,Neutral,Sentiment
date,,,,,,,
2010-06-29,,,0.0000,0.000,0.000,0.000,neutral
2010-06-30,,,0.0000,0.000,0.000,0.000,neutral
2010-07-01,,,0.0000,0.000,0.000,0.000,neutral
2010-07-02,,,0.0000,0.000,0.000,0.000,neutral
2010-07-06,,,0.0000,0.000,0.000,0.000,neutral
...,...,...,...,...,...,...,...
2021-01-13,"@NASASpaceflight Sea shanty tiktok takes it to a new level with actual pirate ships @ConnectDotsToo Sure Today at SpaceX is about practicing Starship engine starts. Ship is held down by massive pins while engines are fired. Two starts completed, about to try a third. @Tesmanian_com As promised Legalize comedy @lvladimirovBG You can steal our name/logos &amp; we probably won’t sue you @arstechnica @SciGuySpace We’re just trying to get people to Mars. Help would be appreciated. @lexfridman Suicide is more than double homicide https://t.co/MvWGPs9uQE @lexfridman Absolutely Hey you … Yeah you Queen … You’re gonna make it! 💕💕 https://t.co/LrqdIrbIyd @TheOldManPar @DJSnM @MachinePix Hate to say it, but might be true","Sea shanty tiktok takes it to a new level with actual pirate ships Sure Today at SpaceX is about practicing Starship engine starts. Ship is held down by massive pins while engines are fired. Two starts completed, about to try a third. As promised Legalize comedy You can steal our name/logos &amp; we probably won’t sue you We’re just trying to get people to Mars. Help would be appreciated. Suicide is more than double homicide Absolutely Hey you … Yeah you Queen … You’re gonna make it! 💕💕 Hate to say it, but might be true",0.5226,0.141,0.090,0.769,positive
2021-01-14,"@skorusARK Prototypes are easy, volume production is hard, positive cash flow is excruciating @Tesla Physics @Erdayastronaut Detanking &amp; inspections now. Good progress towards our “Hop in &amp; go to Mars!” goal. All three static fires completed &amp; no RUDs! @justpaulinelol @Erdayastronaut @SpaceX Wow, a lot has happened in 10 years! @OfficialJlipper Fair enough haha https://t.co/ho7yGXAS3a","Prototypes are easy, volume production is hard, positive cash flow is excruciating Physics Detanking &amp; inspections now. Good progress towards our “Hop in &amp; go to Mars!” goal. All three static fires completed &amp; no RUDs! Wow, a lot has happened in 10 years! Fair enough haha",0.9357,0.336,0.120,0.545,positive
2021-01-15,"@SuperclusterHQ @w00ki33 Fallout New Texas @Breedlove22 @benmezrich Only Chuck Norris can divide by zero @Cerberu21014829 @Breedlove22 @benmezrich Good point @Breedlove22 @benmezrich The thing we call money is just an information system for labor allocation. What actually matters is making goods &amp; providing services. We should look at currencies from an information theory standpoint. Whichever has least error &amp; latency will win. Monty Python is amazing https://t.co/UJq94IWT88 @RationalEtienne @tobyliiiiiiiiii @Erdayastronaut They sure can twist the knife in that show! @tobyliiiiiiiiii @Erdayastronaut Probably wise @Erdayastronaut We’re making major improvements to ease of engine swap. Needs to be a few hours at most. @PPathole @johnkrausphotos @SpaceX Two of the engines need slight repairs, so will be switched out @johnkrausphotos @SpaceX Nice shot @realOmarAbdalah We don’t have high school internships, but please apply when you’re in college!","Fallout New Texas Only Chuck Norris can divide by zero Good point The thing we call money is just an information system for labor allocation. What actually matters is making goods &amp; providing services. We should look at currencies from an information theory standpoint. Whichever has least error &amp; latency will win. Monty Python is amazing They sure can twist the knife in that show! Probably wise We’re making major improvements to ease of engine swap. Needs to be a few hours at most. Two of the engines need slight repairs, so will be switched out Nice shot We don’t have high school internships, but please apply when you’re in college!",0

In [10]:
# Get dummy variables for Sentiment column
tweets_data['Sentiment'] = pd.get_dummies(tweets_data.Sentiment)
tweets_data.head()

,tweet,cleaned_tweet,Compound,Positive,Negative,Neutral,Sentiment
date,,,,,,,
2010-06-29,,,0.0,0.0,0.0,0.0,0
2010-06-30,,,0.0,0.0,0.0,0.0,0
2010-07-01,,,0.0,0.0,0.0,0.0,0
2010-07-02,,,0.0,0.0,0.0,0.0,0
2010-07-06,,,0.0,0.0,0.0,0.0,0


# Tokenizer

In [11]:
# Imports
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import string
import re

lemmatizer = WordNetLemmatizer()
sw_addon = stopwords.words('english')
sw_addon.append("u")
sw_addon.append("it'")
sw_addon.append("char")
sw_addon.append("’")
sw_addon.append("…")
sw_addon.append("”")
sw_addon.append('“')
sw_addon.append('”')
sw_addon.append('amp')
sw_addon.append('would')
sw_addon.append("n't")
sw_addon.append('wo')

sw_addon = set(sw_addon)


In [12]:
# Tokenize tweets
def tokenizer(text):
    """Tokenizes text."""   
    # Create a list of the words
    words = word_tokenize(text)
    # Convert the words to lowercase
    words = list(filter(lambda w: w.lower(), words))   
    # Remove the punctuation
    words = list(filter(lambda t: t not in punctuation, words))   
    # Remove the stopwords
    words = list(filter(lambda t: t.lower() not in sw_addon, words))   
    # Lemmatize Words into root words
    tokens = [lemmatizer.lemmatize(word) for word in words]
    
    return tokens

In [13]:
# Add the tokens column to the Tweets DataFrame
tweets_data["tokens"] = tweets_data.cleaned_tweet.apply(tokenizer)
tweets_data.tail(5)

,tweet,cleaned_tweet,Compound,Positive,Negative,Neutral,Sentiment,tokens
date,,,,,,,,
2021-01-13,"@NASASpaceflight Sea shanty tiktok takes it to a new level with actual pirate ships @ConnectDotsToo Sure Today at SpaceX is about practicing Starship engine starts. Ship is held down by massive pins while engines are fired. Two starts completed, about to try a third. @Tesmanian_com As promised Legalize comedy @lvladimirovBG You can steal our name/logos &amp; we probably won’t sue you @arstechnica @SciGuySpace We’re just trying to get people to Mars. Help would be appreciated. @lexfridman Suicide is more than double homicide https://t.co/MvWGPs9uQE @lexfridman Absolutely Hey you … Yeah you Queen … You’re gonna make it! 💕💕 https://t.co/LrqdIrbIyd @TheOldManPar @DJSnM @MachinePix Hate to say it, but might be true","Sea shanty tiktok takes it to a new level with actual pirate ships Sure Today at SpaceX is about practicing Starship engine starts. Ship is held down by massive pins while engines are fired. Two starts completed, about to try a third. As promised Legalize comedy You can steal our name/logos &amp; we probably won’t sue you We’re just trying to get people to Mars. Help would be appreciated. Suicide is more than double homicide Absolutely Hey you … Yeah you Queen … You’re gonna make it! 💕💕 Hate to say it, but might be true",0.5226,0.141,0.090,0.769,0,"[Sea, shanty, tiktok, take, new, level, actual, pirate, ship, Sure, Today, SpaceX, practicing, Starship, engine, start, Ship, held, massive, pin, engine, fired, Two, start, completed, try, third, promised, Legalize, comedy, steal, name/logos, probably, sue, trying, get, people, Mars, Help, appreciated, Suicide, double, homicide, Absolutely, Hey, Yeah, Queen, gon, na, make, 💕💕, Hate, say, might, true]"
2021-01-14,"@skorusARK Prototypes are easy, volume production is hard, positive cash flow is excruciating @Tesla Physics @Erdayastronaut Detanking &amp; inspections now. Good progress towards our “Hop in &amp; go to Mars!” goal. All three static fires completed &amp; no RUDs! @justpaulinelol @Erdayastronaut @SpaceX Wow, a lot has happened in 10 years! @OfficialJlipper Fair enough haha https://t.co/ho7yGXAS3a","Prototypes are easy, volume production is hard, positive cash flow is excruciating Physics Detanking &amp; inspections now. Good progress towards our “Hop in &amp; go to Mars!” goal. All three static fires completed &amp; no RUDs! Wow, a lot has happened in 10 years! Fair enough haha",0.9357,0.336,0.120,0.545,0,"[Prototypes, easy, volume, production, hard, positive, cash, flow, excruciating, Physics, Detanking, inspection, Good, progress, towards, Hop, go, Mars, goal, three, static, fire, completed, RUDs, Wow, lot, happened, 10, year, Fair, enough, haha]"
2021-01-15,"@SuperclusterHQ @w00ki33 Fallout New Texas @Breedlove22 @benmezrich Only Chuck Norris can divide by zero @Cerberu21014829 @Breedlove22 @benmezrich Good point @Breedlove22 @benmezrich The thing we call money is just an information system for labor allocation. What actually matters is making goods &amp; providing services. We should look at currencies from an information theory standpoint. Whichever has least error &amp; latency will win. Monty Python is amazing https://t.co/UJq94IWT88 @RationalEtienne @tobyliiiiiiiiii @Erdayastronaut They sure can twist the knife in that show! @tobyliiiiiiiiii @Erdayastronaut Probably wise @Erdayastronaut We’re making major improvements to ease of engine swap. Needs to be a few hours at most. @PPathole @johnkrausphotos @SpaceX Two of the engines need slight repairs, so will be switched out @johnkrausphotos @SpaceX Nice shot @realOmarAbdalah We don’t have high school internships, but please apply when you’re in college!","Fallout New Texas Only Chuck Norris can divide by zero Good point The thing we call money is just an information system for labor allocation. What actually matters is making goods &amp; providing services. We should look at currencies from an information theory st

In [14]:
# Save the Tweets DataFrame with the tokens as a csv
file_name="tokens_n_vader_sentiment.csv"
output_file = Path(f"../Resources/{file_name}")
tweets_data.to_csv(f"{output_file}")

## NGrams and Frequency Analysis

In [15]:
# Imports
from collections import Counter
from nltk import ngrams
import inspect

In [16]:
def seriesToList(s):
    """"Converts series into one list"""
    
    lst = []      
    # traverse in the lists   
    for ele in s:  
        lst += ele     
    # return list   
    return lst

tweets = seriesToList(tweets_data["tokens"])

In [17]:
# add to stopwords: amp, would, "n't", 'wo'
# keywords: model 3, coming soon, next week, next year, falcon 9, falcon heavy, model x, 
# next month, space station, sustainable energy, electric car, climate change, software update, dual motor, self-driving, giga berlin, rocket engine, solar power

bigrams = ngrams(tweets, n=2)
tweets_dict = dict(Counter(bigrams).most_common(10))
tweets_bigrams = tweets_dict.items()
list(tweets_bigrams)

[(('Model', '3'), 176),
 (('Falcon', '9'), 67),
 (('next', 'week'), 60),
 (('next', 'year'), 60),
 (('coming', 'soon'), 49),
 (('Falcon', 'Heavy'), 48),
 (('Model', 'X'), 47),
 (('Good', 'point'), 43),
 (('Coming', 'soon'), 41),
 (('year', 'ago'), 40)]

In [18]:
keywords = ['model 3', 'coming soon', 'next week', 'next year', 'falcon 9', 'falcon heavy', 
            'model x', 'next month', 'space station', 'sustainable energy', 'electric car', 'climate change', 
            'software update', 'dual motor', 'self-driving', 'giga berlin', 'rocket', 'engine', 'solar power', 'SpaceX', 
           ]
keywords_df = pd.DataFrame(keywords)
keywords_df

,0
0,model 3
1,coming soon
2,next week
3,next year
4,falcon 9
5,falcon heavy
6,model x
7,next month
8,space station
9,sustainable energy


In [19]:
# keywords: Tesla, car, Model, 3, engine, spacex, launch, production, Falcon, Starship, Mars, 
# maybe?: time, rocket, soon, 
keywords1 = ['launch', 'production', 'starship', 'mars', 'time', 'rocket', 'soon', 'tesla', 'spacex']

bigrams = ngrams(tweets, n=1)
tweets_dict = dict(Counter(bigrams).most_common(50))
tweets_bigrams = tweets_dict.items()
list(tweets_bigrams)

[(('Tesla',), 1058),
 (('car',), 543),
 (('Yes',), 479),
 (('year',), 398),
 (('Model',), 397),
 (('like',), 390),
 (('time',), 333),
 (('good',), 317),
 (('make',), 314),
 (('3',), 309),
 (('much',), 294),
 (('need',), 282),
 (('one',), 261),
 (('rocket',), 255),
 (('great',), 254),
 (('work',), 248),
 (('people',), 242),
 (('get',), 238),
 (("'s",), 237),
 (('next',), 231),
 (('high',), 231),
 (('soon',), 227),
 (('SpaceX',), 222),
 (('Yeah',), 221),
 (('engine',), 211),
 (('right',), 208),
 (('week',), 206),
 (('way',), 204),
 (('day',), 201),
 (('launch',), 198),
 (('better',), 196),
 (('production',), 194),
 (('go',), 190),
 (('month',), 189),
 (('team',), 187),
 (('Falcon',), 182),
 (('test',), 179),
 (('Exactly',), 176),
 (('Good',), 175),
 (('many',), 173),
 (('Thanks',), 171),
 (('point',), 170),
 (('Starship',), 168),
 (('Mars',), 166),
 (('even',), 164),
 (('think',), 163),
 (('first',), 161),
 (('actually',), 161),
 (('lot',), 158),
 (('new',), 157)]

In [20]:
def token_count(tokens, N=15):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [21]:
token_count(tweets)

[('Tesla', 1058),
 ('car', 543),
 ('Yes', 479),
 ('year', 398),
 ('Model', 397),
 ('like', 390),
 ('time', 333),
 ('good', 317),
 ('make', 314),
 ('3', 309),
 ('much', 294),
 ('need', 282),
 ('one', 261),
 ('rocket', 255),
 ('great', 254)]